
# Cross‑Cultural Success Dashboard (Notebook)

Run the **Hybrid Framework** notebook first so that it creates CSVs in `/mnt/data/artifacts/`.
Then run this notebook to visualize and export a static HTML dashboard.


In [ ]:

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

ARTIFACTS = Path('/mnt/data/artifacts')
ARTIFACTS.mkdir(parents=True, exist_ok=True)
print('Artifacts path:', ARTIFACTS)


In [ ]:

import pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

ARTIFACTS = Path('/mnt/data/artifacts')

by_country = pd.read_csv(ARTIFACTS/'metrics_by_country.csv')
by_region  = pd.read_csv(ARTIFACTS/'metrics_by_region.csv')
by_inst    = pd.read_csv(ARTIFACTS/'metrics_by_institution.csv')
cal        = pd.read_csv(ARTIFACTS/'calibration_curve.csv')
cm         = pd.read_csv(ARTIFACTS/'confusion_matrix.csv', index_col=0)
scored     = pd.read_csv(ARTIFACTS/'scored_students.csv')

display(by_country.head(), by_region.head(), by_inst.head())


In [ ]:

import matplotlib.pyplot as plt
def barplot(df, x, y, title, fname):
    plt.figure()
    df_sorted = df.sort_values(y, ascending=False)
    plt.bar(df_sorted[x].astype(str), df_sorted[y])
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(y)
    plt.title(title)
    plt.tight_layout()
    plt.savefig((ARTIFACTS/fname).as_posix())
    plt.show()

barplot(by_country, 'country', 'auroc', 'AUROC by Country', 'auroc_by_country.png')
barplot(by_region,  'region', 'auroc',  'AUROC by Region',  'auroc_by_region.png')
barplot(by_inst,    'institution', 'auroc','AUROC by Institution','auroc_by_inst.png')


In [ ]:

# Positive prediction rate by country
plt.figure()
df = by_country.sort_values('pos_rate', ascending=False)
plt.bar(df['country'].astype(str), df['pos_rate'])
plt.xticks(rotation=45, ha='right')
plt.ylabel('Positive Rate (flag rate)')
plt.title('Selection Rate by Country')
plt.tight_layout()
plt.savefig((ARTIFACTS/'pos_rate_by_country.png').as_posix())
plt.show()


In [ ]:

# Calibration
plt.figure()
plt.plot(cal['prob_pred_bin_mean'], cal['frac_pos'], marker='o')
plt.plot([0,1],[0,1],'--')
plt.xlabel('Predicted probability (bin mean)')
plt.ylabel('Fraction of positives')
plt.title('Calibration Curve')
plt.tight_layout()
plt.savefig((ARTIFACTS/'calibration_curve.png').as_posix())
plt.show()


In [ ]:

# Top at-risk
top = scored.sort_values('y_prob', ascending=False).head(50)
top.to_csv(ARTIFACTS/'top_at_risk.csv', index=False)
top.head(10)


In [ ]:

# Export static HTML
import pandas as pd
html = f'''
<html><head><meta charset="utf-8"><title>Cross‑Cultural Success Dashboard</title></head>
<body style="font-family: Arial, sans-serif; padding: 16px;">
<h1>Cross‑Cultural Success Dashboard</h1>
<h2>Performance</h2>
<img src="auroc_by_country.png" width="700">
<img src="auroc_by_region.png" width="700">
<img src="auroc_by_inst.png" width="700">
<h2>Fairness</h2>
<img src="pos_rate_by_country.png" width="700">
<h2>Calibration</h2>
<img src="calibration_curve.png" width="700">
<h2>Top At‑Risk (first 10)</h2>
{pd.read_csv(ARTIFACTS/'top_at_risk.csv').head(10).to_html(index=False)}
</body></html>
'''
out_path = ARTIFACTS/'dashboard.html'
with open(out_path, 'w', encoding='utf-8') as f:
    f.write(html)
out_path.as_posix()
